In [3]:
import pandas as pd
import os
import numpy as np

In [17]:
from sklearn.metrics import accuracy_score as acs
from sklearn.metrics import confusion_matrix as cm
from sklearn.model_selection import train_test_split as tts

In [ ]:
os.chdir('c://users//administrator//desktop')

In [4]:
t2 = pd.read_csv('Task_2_train.csv')

In [7]:
#t2 = t2.drop(['Unnamed: 0'],axis =1)
t2.head()

,news_text,news_number,news_type
0,US bloggers banned from entering UK,111111112,non-propaganda
1,NaN,111111112,non-propaganda
2,Two prominent US bloggers have been banned fro...,111111112,non-propaganda
3,NaN,111111112,non-propaganda
4,Pamela Geller and Robert Spencer co-founded an...,111111112,propaganda


In [11]:
#t2 = t2.set_index('news_number')
t2.shape

(15170, 2)

In [12]:
t2 = t2.dropna(axis=0)
t2.shape

(14263, 2)

In [14]:
np.sum(t2.isna())

news_text    0
news_type    0
dtype: int64

In [15]:
x = t2['news_text']
y = t2['news_type']

In [19]:
x_tr,x_ts,y_tr,y_ts = tts(x,y,test_size =0.25)

In [ ]:
##vec

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [22]:
tfidfv = TfidfVectorizer(stop_words='english', max_df=0.4)

In [23]:
x_bw_m = tfidfv.fit(x_tr)

In [24]:
x_tr_tfm = x_bw_m.transform(x_tr)
x_ts_tfm = x_bw_m.transform(x_ts)

In [ ]:
#KERAS

In [25]:
from keras.models import Sequential
from keras import layers

dim = x_tr_tfm.shape[1]  # Number of features

sq = Sequential()
sq.add(layers.Dense(10, input_dim=dim, activation='relu'))
sq.add(layers.Dense(1, activation='sigmoid'))

C:\Users\Administrator\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [26]:
sq.compile(loss='binary_crossentropy', 
             optimizer='adam', 
              metrics=['accuracy'])
sq.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                164920    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 164,931
Trainable params: 164,931
Non-trainable params: 0
_________________________________________________________________


In [27]:
d = {'non-propaganda':0,'propaganda':1}

y_tr1 = y_tr.map(d)
y_ts1 = y_ts.map(d)

In [45]:
model_fit = sq.fit(x_tr_tfm, y_tr1, epochs=5,verbose=False, validation_data=(x_ts_tfm, y_ts1),batch_size=3)

In [44]:
loss, accuracy = sq.evaluate(x_tr_tfm, y_tr1, verbose=False)

print("Training Accuracy: {:.4f}".format(accuracy))


loss, accuracy = sq.evaluate(x_ts_tfm, y_ts1, verbose=False)

print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9903
Testing Accuracy:  0.7019


In [46]:
y_tsp = sq.predict_classes(x_ts_tfm)

In [47]:
cm(y_ts1,y_tsp)

array([[2105,  499],
       [ 555,  407]], dtype=int64)

In [48]:
tn,fp,fn,tp = cm(y_ts1,y_tsp).ravel()
tn,fp,fn,tp

(2105, 499, 555, 407)

In [49]:
rec = (tp/(tp+fn))             # recall/sensitivity
pre = (tp/(tp+fp))             # precision
f1_ = (2*pre*rec)/(pre+rec)

print(f1_k)

0.3358662613981763


In [50]:
from sklearn.metrics import classification_report as cf

rep = cf(y_ts1,y_tsp)
rep

'             precision    recall  f1-score   support\n\n          0       0.79      0.81      0.80      2604\n          1       0.45      0.42      0.44       962\n\navg / total       0.70      0.70      0.70      3566\n'